# [sklearn] SVM기반 분류모델_뇌졸중,중풍여부(지체장애인 유의미한 변수_오버샘플링) 1차 시도

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.svm import SVC
import pickle
import os
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
df = pd.read_excel("pydata/[로지스틱] 유의한 변수 추출(지체장애 뇌졸중, 중풍)ver1.xlsx")
df.head()

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,...,16)만성질환명(골관절염(퇴행성_x000D_\n관절염)),17)만성질환명(류마티스 관절염),18)만성질환명(골다공증),19)만성질환명(척추측만증),20)만성질환명(허리목통증),21)만성질환명(피부염),22)만성질환명(백내장),23)만성질환명(우울증),24)만성질환명(암),25)만성질환명(기타)
0,1970,250,0,6,21,36,64,12,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1949,150,30,6,21,14,69,8,1,0,...,0,0,0,0,0,0,0,0,0,1
2,1958,200,0,5,11,38,48,11,0,0,...,0,0,0,0,1,0,0,0,0,1
3,1955,154,30,5,29,38,48,11,1,0,...,0,0,0,0,1,0,0,1,0,0
4,1936,270,0,3,24,36,48,6,0,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
df.columns

Index(['생년', '가구 월평균 지출액', '월 혈압약 일수', '장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
       '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부',
       '01)만성질환명(고혈압)', '02)만성질환명(뇌졸중,중풍)', '03)만성질환명(심근경색증)', '04)만성질환명(협심증)',
       '05)만성질환명(이상지혈증)', '06)만성질환명(당뇨병)', '07)만성질환명(갑상선장애)', '08)만성질환명(천식)',
       '09)만성질환명(폐결핵)', '10)만성질환명(폐질환(만성기관 지염,폐기종)', '11)만성질환명(위십이지장궤양)',
       '12)만성질환명(B형간염)', '13)만성질환명(C형간염)', '14)만성질환명(간경변증)', '15)만성질환명(신부전)',
       '16)만성질환명(골관절염(퇴행성_x000D_\n관절염))', '17)만성질환명(류마티스 관절염)',
       '18)만성질환명(골다공증)', '19)만성질환명(척추측만증)', '20)만성질환명(허리목통증)', '21)만성질환명(피부염)',
       '22)만성질환명(백내장)', '23)만성질환명(우울증)', '24)만성질환명(암)', '25)만성질환명(기타)'],
      dtype='object')

In [6]:
df1=df[['생년', '가구 월평균 지출액', '월 혈압약 일수', '장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
       '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부', '02)만성질환명(뇌졸중,중풍)']]
df1 #2686건 

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
0,1970,250,0,6,21,36,64,12,0,0,0,1,1,0
1,1949,150,30,6,21,14,69,8,1,0,0,1,2,0
2,1958,200,0,5,11,38,48,11,0,0,0,1,1,0
3,1955,154,30,5,29,38,48,11,1,0,0,1,1,0
4,1936,270,0,3,24,36,48,6,0,0,0,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,1935,100,0,3,11,36,68,12,0,0,0,1,2,0
2682,1952,100,0,5,11,38,69,12,0,0,0,1,1,0
2683,1960,200,30,6,11,38,48,11,1,0,0,1,1,0
2684,1960,200,0,6,11,31,48,11,0,0,0,1,1,0


### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [5]:
df1[df1['02)만성질환명(뇌졸중,중풍)']==1]

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
39,1928,100,30,6,40,36,48,6,1,0,0,3,2,1
93,1938,200,0,5,21,38,64,11,0,0,0,1,1,1
102,1938,100,30,5,11,36,48,6,1,0,0,2,2,1
126,1932,180,0,5,39,36,48,6,0,0,0,2,2,1
147,1930,500,30,1,32,46,14,14,1,0,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,1979,150,30,4,21,36,62,12,1,0,0,1,2,1
2570,1939,200,30,5,11,21,80,13,1,8,0,1,1,1
2576,1965,250,0,5,21,38,69,12,0,0,0,1,1,1
2588,1930,80,0,4,40,24,40,14,0,0,0,2,2,1


In [7]:
out=df1.loc[[147]] # 유: 39, 93, 147 / 무: 3, 10
out

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
147,1930,500,30,1,32,46,14,14,1,0,0,2,2,1


In [8]:
df2=df1.drop(out.index).reset_index(drop=True) # 39번 인덱스 제거
df2.head()

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
0,1970,250,0,6,21,36,64,12,0,0,0,1,1,0
1,1949,150,30,6,21,14,69,8,1,0,0,1,2,0
2,1958,200,0,5,11,38,48,11,0,0,0,1,1,0
3,1955,154,30,5,29,38,48,11,1,0,0,1,1,0
4,1936,270,0,3,24,36,48,6,0,0,0,3,2,0


In [9]:
# 독립변수 테이블 생성
x_data=df2[['생년', '가구 월평균 지출액', '월 혈압약 일수', '장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
       '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']]

In [10]:
# 종족변수 테이블 생성
y_data=df2[['02)만성질환명(뇌졸중,중풍)']]

In [11]:
y_data.value_counts() # 유무 빈도 확인

02)만성질환명(뇌졸중,중풍)
0                   2539
1                    146
dtype: int64

In [12]:
# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777, stratify=y_data)

In [45]:
y_test.value_counts()

02)만성질환명(뇌졸중,중풍)
0                   762
1                    44
dtype: int64

In [46]:
# 범주형 변수: 원핫 인코딩 수행(MaxAbsScaler: 최대절대값과 0이 각각 1, 0이 되도록 스케일링. 즉, 즉 -1~1 사이로 재조정)
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
#        '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_train)
# x_train = transformer.transform(x_train)
# x_test = transformer.transform(x_test)

In [13]:
#모델 생성
model = SVC(kernel='rbf', C=5.0, gamma=0.1, probability=True)#,class_weight=class_weight)

In [14]:
#모델 학습
model.fit(x_train, y_train)

#모델 검증
print('training data fit :', model.score(x_train, y_train)) 
print('test data fit :', model.score(x_test, y_test)) 

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(model, f)

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training data fit : 0.9973390101117616
test data fit : 0.9267990074441688


In [15]:
x_test=out.drop(['02)만성질환명(뇌졸중,중풍)'],axis=1) # out 데이터에서 '뇌졸중, 중풍' 칼럼만 제거
x_test

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부
147,1930,500,30,1,32,46,14,14,1,0,0,2,2


In [16]:
labels = ['무', '유']

In [18]:
# x_test = transformer.transform(x_test)

y_predict = model.predict(x_test)
label = labels[y_predict[0]]
y_predict = model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.9444249593113747


---

## 오버샘플링_SVC

### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [19]:
df1[df1['02)만성질환명(뇌졸중,중풍)']==1]

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
39,1928,100,30,6,40,36,48,6,1,0,0,3,2,1
93,1938,200,0,5,21,38,64,11,0,0,0,1,1,1
102,1938,100,30,5,11,36,48,6,1,0,0,2,2,1
126,1932,180,0,5,39,36,48,6,0,0,0,2,2,1
147,1930,500,30,1,32,46,14,14,1,0,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,1979,150,30,4,21,36,62,12,1,0,0,1,2,1
2570,1939,200,30,5,11,21,80,13,1,8,0,1,1,1
2576,1965,250,0,5,21,38,69,12,0,0,0,1,1,1
2588,1930,80,0,4,40,24,40,14,0,0,0,2,2,1


In [20]:
out=df1.loc[[126]] # 유: 39, 93, 126 / 무: 3, 10, 100
out

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
126,1932,180,0,5,39,36,48,6,0,0,0,2,2,1


In [21]:
df2=df1.drop(out.index).reset_index(drop=True) # 39번 인덱스 제거
df2.head()

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
0,1970,250,0,6,21,36,64,12,0,0,0,1,1,0
1,1949,150,30,6,21,14,69,8,1,0,0,1,2,0
2,1958,200,0,5,11,38,48,11,0,0,0,1,1,0
3,1955,154,30,5,29,38,48,11,1,0,0,1,1,0
4,1936,270,0,3,24,36,48,6,0,0,0,3,2,0


In [22]:
# 독립변수 테이블 생성
x_data=df2[['생년', '가구 월평균 지출액', '월 혈압약 일수', '장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
       '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']]

In [23]:
# 종족변수 테이블 생성
y_data=df2[['02)만성질환명(뇌졸중,중풍)']]

In [24]:
y_data.value_counts() # 유무 빈도 확인

02)만성질환명(뇌졸중,중풍)
0                   2539
1                    146
dtype: int64

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [208]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [30]:
from imblearn.over_sampling import SMOTE, ADASYN

# smote = SMOTE()
# # adasyn = ADASYN()
# x_sm, y_sm = smote.fit_resample(x_train, y_train)

# y_sm.value_counts() # 비율 1:1 확인

In [210]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [142]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
#        '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_train)
# x_scale = transformer.transform(x_train)
# x_scale = transformer.transform(x_test)

In [26]:
#모델 생성
model = SVC(kernel='rbf', C=5.0, gamma=0.1, probability=True)#,class_weight=class_weight)

#모델 학습
model.fit(x_train, y_train)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(model.score(x_train, y_train)))
print('평가 데이터 점수 :{:.4f}'.format(model.score(x_test, y_test))) 

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


학습 데이터 점수 :0.9957
평가 데이터 점수 :0.9442


In [27]:
numeric_features = x_train.columns.tolist()[:3] 
# numeric_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())])

categorical_features = x_train.columns.tolist()[3:]
# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])
# clf = Pipeline(steps=[('preprocessor', preprocessor),])

In [28]:
column_transformer = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], 
    remainder="passthrough")

In [31]:
# 그리드 서치
from imblearn.pipeline import Pipeline # 파이프라인 구축 시, smote 함수를 사용할 때 불러오는 패키지
pipeline = Pipeline([('processor', column_transformer), 
                    ('over', SMOTE()), 
                    ('model', SVC(probability=True))])

## 비대칭 매개변수 그리드 탐색
param_grid = [ {'model__kernel': ['rbf', 'ploy', 'sigmoid'], # 가장 성능이 좋은 것: 가우시안 RBF 커널
             'model__C': [0.01, 0.1, 1, 10], # 값이 클수록 이상치의 존재 가능성을 작게 봐서 좀 더 세심하게 결정 경계를 찾아낸다. -> 오버피팅(하드마진)
             'model__gamma': [0.01, 0.1, 1, 10]},  # 값이 클수록 결정 경계의 곡률이 커짐 -> 오버피팅
             {'model__kernel': ['linear'],
              'model__C': [0.01, 0.1, 1, 10]
             } ]

gs = GridSearchCV(estimator=pipeline, 
                    param_grid=param_grid, scoring='accuracy', cv = 5, 
                    n_jobs=-1)

result = gs.fit(x_train, y_train)

print('최적 파라미터: {}'.format(gs.best_params_))
print('최적 점수: {:.4f}'.format(gs.best_score_))
print(result.best_estimator_)

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
80 fits failed out of a total of 260.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\imblearn\pipeline.py", line 272, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
 

최적 파라미터: {'model__C': 0.01, 'model__gamma': 10, 'model__kernel': 'rbf'}
최적 점수: 0.9292
Pipeline(steps=[('processor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['생년', '가구 월평균 지출액',
                                                   '월 혈압약 일수']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['장애등급', '가구 유형', '가장 불편한부위',
                                                   '장애주된 원인', '주된 진단명',
                                                   '혈압약 복용 여부', '암 종류1',
                                                   '암 종류2', 'EQ-5D (자기관리)',
                                                   '스마트폰 사용 여부'])])),
                ('over', SMOTE()),
                ('model', SVC(C=0.01, gamma=10, probability=

In [32]:
#모델 학습
svc_model = gs.best_estimator_

In [33]:
x_test=out.drop(['02)만성질환명(뇌졸중,중풍)'],axis=1)
x_test

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부
126,1932,180,0,5,39,36,48,6,0,0,0,2,2


In [34]:
labels = ['무', '유']

In [35]:
# x_test = transformer.transform(x_test)

y_predict = svc_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = svc_model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.5


---
## 오버샘플링_KNN

In [153]:
from sklearn.neighbors import KNeighborsClassifier

### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [154]:
df1[df1['02)만성질환명(뇌졸중,중풍)']==1]

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
39,1928,100,30,6,40,36,48,6,1,0,0,3,2,1
93,1938,200,0,5,21,38,64,11,0,0,0,1,1,1
102,1938,100,30,5,11,36,48,6,1,0,0,2,2,1
126,1932,180,0,5,39,36,48,6,0,0,0,2,2,1
147,1930,500,30,1,32,46,14,14,1,0,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,1979,150,30,4,21,36,62,12,1,0,0,1,2,1
2570,1939,200,30,5,11,21,80,13,1,8,0,1,1,1
2576,1965,250,0,5,21,38,69,12,0,0,0,1,1,1
2588,1930,80,0,4,40,24,40,14,0,0,0,2,2,1


In [155]:
out=df1.loc[[2570]] # 유: 39, 93, 126 / 무: 3, 10, 100
out

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
2570,1939,200,30,5,11,21,80,13,1,8,0,1,1,1


In [156]:
df2=df1.drop(out.index).reset_index(drop=True) # 39번 인덱스 제거
df2.head()

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
0,1970,250,0,6,21,36,64,12,0,0,0,1,1,0
1,1949,150,30,6,21,14,69,8,1,0,0,1,2,0
2,1958,200,0,5,11,38,48,11,0,0,0,1,1,0
3,1955,154,30,5,29,38,48,11,1,0,0,1,1,0
4,1936,270,0,3,24,36,48,6,0,0,0,3,2,0


In [157]:
# 독립변수 테이블 생성
x_data=df2[['생년', '가구 월평균 지출액', '월 혈압약 일수', '장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
       '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']]

In [158]:
# 종족변수 테이블 생성
y_data=df2[['02)만성질환명(뇌졸중,중풍)']]

In [159]:
y_data.value_counts() # 유무 빈도 확인

02)만성질환명(뇌졸중,중풍)
0                   2539
1                    146
dtype: int64

In [160]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [161]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [162]:
from imblearn.over_sampling import SMOTE, ADASYN

# smote = SMOTE()
# # adasyn = ADASYN()
# x_sm, y_sm = smote.fit_resample(x_train, y_train)

# y_sm.value_counts() # 비율 1:1 확인

In [163]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [164]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
#        '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_train)
# x_scale = transformer.transform(x_train)
# x_scale = transformer.transform(x_test)

In [165]:
#모델 생성
knn_model = KNeighborsClassifier()

#모델 학습
knn_model.fit(x_train, y_train)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(knn_model.score(x_train, y_train)))
print('평가 데이터 점수 :{:.4f}'.format(knn_model.score(x_test, y_test))) 

C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


학습 데이터 점수 :0.9409
평가 데이터 점수 :0.9578


In [166]:
numeric_features = x_train.columns.tolist()[:3] 
# numeric_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())])

categorical_features = x_train.columns.tolist()[3:]
# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])
# clf = Pipeline(steps=[('preprocessor', preprocessor),])

In [167]:
column_transformer = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], 
    remainder="passthrough")

In [168]:
# 그리드 서치
from imblearn.pipeline import Pipeline # 파이프라인 구축 시, smote 함수를 사용할 때 불러오는 패키지
pipeline = Pipeline([('processor', column_transformer), 
                    ('over', SMOTE()), 
                    ('model', KNeighborsClassifier())])

param_grid = {'model__n_neighbors': range(1, 16),
             'model__weights': ['uniform', 'distance'],
             'model__algorithm': ['ball_tree', 'kd_tree', 'brute'],
             'model__p': [1,2] # 1: 멘하튼 거리, 2: 유클리디안 거리
             }

gs = GridSearchCV(estimator=pipeline, param_grid=param_grid, n_jobs=-1, cv = 5)

result = gs.fit(x_train, y_train)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {:.4f}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

최적 파라미터: {'model__algorithm': 'brute', 'model__n_neighbors': 2, 'model__p': 1, 'model__weights': 'uniform'}
최적 점수: 0.9085
Pipeline(steps=[('processor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['생년', '가구 월평균 지출액',
                                                   '월 혈압약 일수']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['장애등급', '가구 유형', '가장 불편한부위',
                                                   '장애주된 원인', '주된 진단명',
                                                   '혈압약 복용 여부', '암 종류1',
                                                   '암 종류2', 'EQ-5D (자기관리)',
                                                   '스마트폰 사용 여부'])])),
                ('over', SMOTE()),
                ('model'

C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [169]:
#모델 학습
knn_model = gs.best_estimator_

In [170]:
x_test=out.drop(['02)만성질환명(뇌졸중,중풍)'],axis=1)
x_test

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부
2570,1939,200,30,5,11,21,80,13,1,8,0,1,1


In [171]:
labels = ['무', '유']

In [172]:
# x_test = transformer.transform(x_test)

y_predict = knn_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = knn_model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 1.0


---
## 오버샘플링_LightGBM

In [140]:
import lightgbm as lgbm

### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [173]:
df1[df1['02)만성질환명(뇌졸중,중풍)']==1]

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
39,1928,100,30,6,40,36,48,6,1,0,0,3,2,1
93,1938,200,0,5,21,38,64,11,0,0,0,1,1,1
102,1938,100,30,5,11,36,48,6,1,0,0,2,2,1
126,1932,180,0,5,39,36,48,6,0,0,0,2,2,1
147,1930,500,30,1,32,46,14,14,1,0,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,1979,150,30,4,21,36,62,12,1,0,0,1,2,1
2570,1939,200,30,5,11,21,80,13,1,8,0,1,1,1
2576,1965,250,0,5,21,38,69,12,0,0,0,1,1,1
2588,1930,80,0,4,40,24,40,14,0,0,0,2,2,1


In [389]:
out=df1.loc[[2000]] # 유: 39, 93, 126 / 무: 3, 10, 100
out

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
2000,1943,100,30,6,11,14,69,8,1,0,0,1,2,0


In [390]:
df2=df1.drop(out.index).reset_index(drop=True) # 39번 인덱스 제거
df2.head()

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
0,1970,250,0,6,21,36,64,12,0,0,0,1,1,0
1,1949,150,30,6,21,14,69,8,1,0,0,1,2,0
2,1958,200,0,5,11,38,48,11,0,0,0,1,1,0
3,1955,154,30,5,29,38,48,11,1,0,0,1,1,0
4,1936,270,0,3,24,36,48,6,0,0,0,3,2,0


In [391]:
# 독립변수 테이블 생성
x_data=df2[['생년', '가구 월평균 지출액', '월 혈압약 일수', '장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
       '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']]

In [392]:
# 종족변수 테이블 생성
y_data=df2[['02)만성질환명(뇌졸중,중풍)']]

In [393]:
y_data.value_counts() # 유무 빈도 확인

02)만성질환명(뇌졸중,중풍)
0                   2538
1                    147
dtype: int64

In [394]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [395]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [396]:
from imblearn.over_sampling import SMOTE, ADASYN

smote = SMOTE()
# # adasyn = ADASYN()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

y_sm.value_counts() # 비율 1:1 확인

02)만성질환명(뇌졸중,중풍)
0                   1765
1                   1765
dtype: int64

In [397]:
print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

After OverSampling, the shape of train_X: (3530, 13)
After OverSampling, the shape of train_y: (3530, 1) 



[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [398]:
transformer = make_column_transformer(
        (OneHotEncoder(handle_unknown = "ignore"), ['장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
       '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']),
        remainder='passthrough')
transformer = make_pipeline(transformer, MaxAbsScaler())
transformer.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

In [399]:
#모델 생성
lgb_model= lgbm.LGBMClassifier()

#모델 학습
lgb_model.fit(x_sm, y_sm)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(lgb_model.score(x_sm, y_sm)))
print('평가 데이터 점수 :{:.4f}'.format(lgb_model.score(x_test, y_test))) 

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


학습 데이터 점수 :0.9768
평가 데이터 점수 :0.8871


In [306]:
numeric_features = x_train.columns.tolist()[:3] 
# numeric_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())])

categorical_features = x_train.columns.tolist()[3:]
# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])
# clf = Pipeline(steps=[('preprocessor', preprocessor),])

In [307]:
column_transformer = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], 
    remainder="passthrough")

In [308]:
# 그리드 서치
from imblearn.pipeline import Pipeline # 파이프라인 구축 시, smote 함수를 사용할 때 불러오는 패키지
pipeline = Pipeline([('processor', column_transformer), 
                    ('over', SMOTE()), 
                    ('model', lgbm.LGBMClassifier())])

param_grid = {'model__learning_rate': [0.05, 0.1, 0.2],
             'model__max_depth': [3, 5, 7]}

gs = GridSearchCV(estimator = pipeline, param_grid=param_grid, n_jobs=-1, cv = 5)

result = gs.fit(x_train, y_train)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {:.4f}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


최적 파라미터: {'model__learning_rate': 0.2, 'model__max_depth': 3}
최적 점수: 0.9266
Pipeline(steps=[('processor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['생년', '가구 월평균 지출액',
                                                   '월 혈압약 일수']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['장애등급', '가구 유형', '가장 불편한부위',
                                                   '장애주된 원인', '주된 진단명',
                                                   '혈압약 복용 여부', '암 종류1',
                                                   '암 종류2', 'EQ-5D (자기관리)',
                                                   '스마트폰 사용 여부'])])),
                ('over', SMOTE()),
                ('model', LGBMClassifier(learning_rate=0.2, max_depth=

In [309]:
#모델 학습
lgb_model = gs.best_estimator_

In [400]:
x_test=out.drop(['02)만성질환명(뇌졸중,중풍)'],axis=1)
x_test

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부
2000,1943,100,30,6,11,14,69,8,1,0,0,1,2


In [401]:
labels = ['무', '유']

In [402]:
x_test = transformer.transform(x_test)

y_predict = lgb_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = lgb_model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.9779941771682368


---
## 오버샘플링_Logistic Regresstion

In [403]:
from sklearn.linear_model import LogisticRegression

### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [404]:
df1[df1['02)만성질환명(뇌졸중,중풍)']==1]

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
39,1928,100,30,6,40,36,48,6,1,0,0,3,2,1
93,1938,200,0,5,21,38,64,11,0,0,0,1,1,1
102,1938,100,30,5,11,36,48,6,1,0,0,2,2,1
126,1932,180,0,5,39,36,48,6,0,0,0,2,2,1
147,1930,500,30,1,32,46,14,14,1,0,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,1979,150,30,4,21,36,62,12,1,0,0,1,2,1
2570,1939,200,30,5,11,21,80,13,1,8,0,1,1,1
2576,1965,250,0,5,21,38,69,12,0,0,0,1,1,1
2588,1930,80,0,4,40,24,40,14,0,0,0,2,2,1


In [461]:
out=df1.loc[[126]] # 유: 39, 93, 126 / 무: 3, 10, 100
out

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
126,1932,180,0,5,39,36,48,6,0,0,0,2,2,1


In [462]:
df2=df1.drop(out.index).reset_index(drop=True) # 39번 인덱스 제거
df2.head()

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
0,1970,250,0,6,21,36,64,12,0,0,0,1,1,0
1,1949,150,30,6,21,14,69,8,1,0,0,1,2,0
2,1958,200,0,5,11,38,48,11,0,0,0,1,1,0
3,1955,154,30,5,29,38,48,11,1,0,0,1,1,0
4,1936,270,0,3,24,36,48,6,0,0,0,3,2,0


In [463]:
# 독립변수 테이블 생성
x_data=df2[['생년', '가구 월평균 지출액', '월 혈압약 일수', '장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
       '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']]

In [464]:
# 종족변수 테이블 생성
y_data=df2[['02)만성질환명(뇌졸중,중풍)']]

In [465]:
y_data.value_counts() # 유무 빈도 확인

02)만성질환명(뇌졸중,중풍)
0                   2539
1                    146
dtype: int64

In [466]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [467]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [468]:
from imblearn.over_sampling import SMOTE, ADASYN

# smote = SMOTE()
# # # adasyn = ADASYN()
# x_sm, y_sm = smote.fit_resample(x_train, y_train)

# y_sm.value_counts() # 비율 1:1 확인

In [439]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [440]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
#        '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_sm)
# x_sm = transformer.transform(x_sm)
# x_test = transformer.transform(x_test)

In [469]:
#모델 생성
log_model= LogisticRegression()

#모델 학습
log_model.fit(x_train, y_train)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(log_model.score(x_train, y_train)))
print('평가 데이터 점수 :{:.4f}'.format(log_model.score(x_test, y_test))) 

학습 데이터 점수 :0.9383
평가 데이터 점수 :0.9628


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [442]:
numeric_features = x_train.columns.tolist()[:3] 
# numeric_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())])

categorical_features = x_train.columns.tolist()[3:]
# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])
# clf = Pipeline(steps=[('preprocessor', preprocessor),])

In [443]:
column_transformer = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], 
    remainder="passthrough")

In [444]:
# 그리드 서치
from imblearn.pipeline import Pipeline # 파이프라인 구축 시, smote 함수를 사용할 때 불러오는 패키지
pipeline = Pipeline([('processor', column_transformer), 
                    ('over', SMOTE()), 
                    ('model', LogisticRegression())])

param_grid = [{'model__penalty' : ['l1', 'l2'],
               'model__C' : [0.001, 0.01, 0.1, 1, 10, 100]}]

gs = GridSearchCV(estimator = pipeline, param_grid=param_grid, n_jobs=-1, cv = 5)

result = gs.fit(x_train, y_train)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {:.4f}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

최적 파라미터: {'model__C': 10, 'model__penalty': 'l2'}
최적 점수: 0.7163
Pipeline(steps=[('processor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['생년', '가구 월평균 지출액',
                                                   '월 혈압약 일수']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['장애등급', '가구 유형', '가장 불편한부위',
                                                   '장애주된 원인', '주된 진단명',
                                                   '혈압약 복용 여부', '암 종류1',
                                                   '암 종류2', 'EQ-5D (자기관리)',
                                                   '스마트폰 사용 여부'])])),
                ('over', SMOTE()), ('model', LogisticRegression(C=10))])


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\imblearn\pipeline.py", line 272, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, se

In [445]:
#모델 학습
log_model = gs.best_estimator_

In [470]:
x_test=out.drop(['02)만성질환명(뇌졸중,중풍)'],axis=1)
x_test

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부
126,1932,180,0,5,39,36,48,6,0,0,0,2,2


In [471]:
labels = ['무', '유']

In [472]:
# x_test = transformer.transform(x_test)

y_predict = log_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = log_model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.9844400593563303


---
## 오버샘플링_Random Forest

In [3]:
from sklearn.ensemble import RandomForestClassifier

### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [7]:
df1[df1['02)만성질환명(뇌졸중,중풍)']==1]

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
39,1928,100,30,6,40,36,48,6,1,0,0,3,2,1
93,1938,200,0,5,21,38,64,11,0,0,0,1,1,1
102,1938,100,30,5,11,36,48,6,1,0,0,2,2,1
126,1932,180,0,5,39,36,48,6,0,0,0,2,2,1
147,1930,500,30,1,32,46,14,14,1,0,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,1979,150,30,4,21,36,62,12,1,0,0,1,2,1
2570,1939,200,30,5,11,21,80,13,1,8,0,1,1,1
2576,1965,250,0,5,21,38,69,12,0,0,0,1,1,1
2588,1930,80,0,4,40,24,40,14,0,0,0,2,2,1


In [8]:
out=df1.loc[[39]] # 유: 39, 93, 126 / 무: 3, 10, 100
out

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
39,1928,100,30,6,40,36,48,6,1,0,0,3,2,1


In [9]:
df2=df1.drop(out.index).reset_index(drop=True) # 39번 인덱스 제거
df2.head()

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
0,1970,250,0,6,21,36,64,12,0,0,0,1,1,0
1,1949,150,30,6,21,14,69,8,1,0,0,1,2,0
2,1958,200,0,5,11,38,48,11,0,0,0,1,1,0
3,1955,154,30,5,29,38,48,11,1,0,0,1,1,0
4,1936,270,0,3,24,36,48,6,0,0,0,3,2,0


In [10]:
# 독립변수 테이블 생성
x_data=df2[['생년', '가구 월평균 지출액', '월 혈압약 일수', '장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
       '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']]

In [11]:
# 종족변수 테이블 생성
y_data=df2[['02)만성질환명(뇌졸중,중풍)']]

In [12]:
y_data.value_counts() # 유무 빈도 확인

02)만성질환명(뇌졸중,중풍)
0                   2539
1                    146
dtype: int64

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [480]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [14]:
from imblearn.over_sampling import SMOTE, ADASYN

# smote = SMOTE()
# # # adasyn = ADASYN()
# x_sm, y_sm = smote.fit_resample(x_train, y_train)

# y_sm.value_counts() # 비율 1:1 확인

In [439]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [440]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
#        '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_sm)
# x_sm = transformer.transform(x_sm)
# x_test = transformer.transform(x_test)

In [15]:
#모델 생성
rfc_model= RandomForestClassifier()

#모델 학습
rfc_model.fit(x_train, y_train)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(rfc_model.score(x_train, y_train)))
print('평가 데이터 점수 :{:.4f}'.format(rfc_model.score(x_test, y_test))) 

<ipython-input-15-92970a86e300>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc_model.fit(x_train, y_train)


학습 데이터 점수 :0.9989
평가 데이터 점수 :0.9591


In [17]:
numeric_features = x_train.columns.tolist()[:3] 
# numeric_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())])

categorical_features = x_train.columns.tolist()[3:]
# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])
# clf = Pipeline(steps=[('preprocessor', preprocessor),])

In [18]:
column_transformer = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], 
    remainder="passthrough")

In [19]:
# 그리드 서치
from imblearn.pipeline import Pipeline # 파이프라인 구축 시, smote 함수를 사용할 때 불러오는 패키지
pipeline = Pipeline([('processor', column_transformer), 
                    ('over', SMOTE()), 
                    ('model', RandomForestClassifier())])

param_grid = {'model__n_estimators': [100, 200, 300, 400, 500],
              'model__max_depth': [3, 5, 7, 9],
              'model__min_samples_split': [3, 5, 7], # 노드를 분할하기 위한 최소한의 샘플 수
              'model__min_samples_leaf': [3, 5, 7]} # 리프 노드가 될 수 있는 샘플 데이터의 최소 개수

gs = GridSearchCV(estimator = pipeline, param_grid=param_grid, n_jobs=-1, cv = 5)

result = gs.fit(x_train, y_train)

print('최적 파라미터: {}'.format(gs.best_params_)) # 최적의 파라미터 출력
print('최적 점수: {:.4f}'.format(gs.best_score_)) # 최적 파라미터의 평가 점수 출력
print(result.best_estimator_) # 최적 파라미터로 학습된 estimator 출력

C:\Users\user\anaconda3\lib\site-packages\imblearn\pipeline.py:272: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, yt, **fit_params_last_step)


최적 파라미터: {'model__max_depth': 9, 'model__min_samples_leaf': 3, 'model__min_samples_split': 5, 'model__n_estimators': 100}
최적 점수: 0.8712
Pipeline(steps=[('processor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['생년', '가구 월평균 지출액',
                                                   '월 혈압약 일수']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['장애등급', '가구 유형', '가장 불편한부위',
                                                   '장애주된 원인', '주된 진단명',
                                                   '혈압약 복용 여부', '암 종류1',
                                                   '암 종류2', 'EQ-5D (자기관리)',
                                                   '스마트폰 사용 여부'])])),
                ('over', SMOTE()),
          

In [23]:
#모델 학습
rfc_model = gs.best_estimator_

In [24]:
x_test=out.drop(['02)만성질환명(뇌졸중,중풍)'],axis=1)
x_test

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부
39,1928,100,30,6,40,36,48,6,1,0,0,3,2


In [25]:
labels = ['무', '유']

In [26]:
# x_test = transformer.transform(x_test)

y_predict = rfc_model.predict(x_test)
label = labels[y_predict[0]]
y_predict = rfc_model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.5318833393188718


---
## 오버샘플링_Soft Voting

In [41]:
df1[df1['02)만성질환명(뇌졸중,중풍)']==1]

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
39,1928,100,30,6,40,36,48,6,1,0,0,3,2,1
93,1938,200,0,5,21,38,64,11,0,0,0,1,1,1
102,1938,100,30,5,11,36,48,6,1,0,0,2,2,1
126,1932,180,0,5,39,36,48,6,0,0,0,2,2,1
147,1930,500,30,1,32,46,14,14,1,0,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,1979,150,30,4,21,36,62,12,1,0,0,1,2,1
2570,1939,200,30,5,11,21,80,13,1,8,0,1,1,1
2576,1965,250,0,5,21,38,69,12,0,0,0,1,1,1
2588,1930,80,0,4,40,24,40,14,0,0,0,2,2,1


In [42]:
out=df1.loc[[39]] # 유: 39, 93, 126 / 무: 3, 10, 100
out

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
39,1928,100,30,6,40,36,48,6,1,0,0,3,2,1


In [43]:
df2=df1.drop(out.index).reset_index(drop=True) # 39번 인덱스 제거
df2.head()

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부,"02)만성질환명(뇌졸중,중풍)"
0,1970,250,0,6,21,36,64,12,0,0,0,1,1,0
1,1949,150,30,6,21,14,69,8,1,0,0,1,2,0
2,1958,200,0,5,11,38,48,11,0,0,0,1,1,0
3,1955,154,30,5,29,38,48,11,1,0,0,1,1,0
4,1936,270,0,3,24,36,48,6,0,0,0,3,2,0


In [44]:
# 독립변수 테이블 생성
x_data=df2[['생년', '가구 월평균 지출액', '월 혈압약 일수', '장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
       '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']]

In [45]:
# 종족변수 테이블 생성
y_data=df2[['02)만성질환명(뇌졸중,중풍)']]

In [46]:
y_data.value_counts() # 유무 빈도 확인

02)만성질환명(뇌졸중,중풍)
0                   2539
1                    146
dtype: int64

In [47]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [48]:
# !pip install SMOTE
# !pip install -U imbalanced-learn

In [49]:
from imblearn.over_sampling import SMOTE, ADASYN

# smote = SMOTE()
# # # adasyn = ADASYN()
# x_sm, y_sm = smote.fit_resample(x_train, y_train)

# y_sm.value_counts() # 비율 1:1 확인

In [50]:
# print('After OverSampling, the shape of train_X: {}'.format(x_sm.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_sm.shape))

[MaxAbsScaler 참고](https://mkjjo.github.io/python/2019/01/10/scaler.html)

In [51]:
# transformer = make_column_transformer(
#         (OneHotEncoder(handle_unknown = "ignore"), ['장애등급', '가구 유형', '가장 불편한부위', '장애주된 원인',
#        '주된 진단명', '혈압약 복용 여부', '암 종류1', '암 종류2', 'EQ-5D (자기관리)', '스마트폰 사용 여부']),
#         remainder='passthrough')
# transformer = make_pipeline(transformer, MaxAbsScaler())
# transformer.fit(x_sm)
# x_sm = transformer.transform(x_sm)
# x_test = transformer.transform(x_test)

In [52]:
numeric_features = x_train.columns.tolist()[:3] 
# numeric_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())])

categorical_features = x_train.columns.tolist()[3:]
# categorical_transformer = Pipeline(steps=[
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)])
# clf = Pipeline(steps=[('preprocessor', preprocessor),])

In [53]:
column_transformer = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], 
    remainder="passthrough")

In [54]:
from sklearn.ensemble import VotingClassifier

In [55]:
# 그리드 서치
from imblearn.pipeline import Pipeline # 파이프라인 구축 시, smote 함수를 사용할 때 불러오는 패키지
pipeline = Pipeline([('processor', column_transformer), 
                    ('over', SMOTE()), 
                    ('model', RandomForestClassifier())])

## Search grid for optimal parameters
rf_param_grid = {"model__max_depth": [None],
              "model__max_features": [3, 8, 8],
              "model__min_samples_split": [2, 3, 8],
              "model__min_samples_leaf": [1, 3, 8],
              "model__bootstrap": [False],
              "model__n_estimators" :[100,300],
              "model__criterion": ["gini"]}
 
gsRFC = GridSearchCV(pipeline, rf_param_grid, cv=5, scoring="accuracy", n_jobs = -1)
#print(score)
 
gsRFC.fit(x_train, y_train)
 
RFC_best = gsRFC.best_estimator_
 
# Best score
gsRFC.best_score_

C:\Users\user\anaconda3\lib\site-packages\imblearn\pipeline.py:272: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, yt, **fit_params_last_step)


0.9281489361702129

In [56]:
# 그리드 서치
from sklearn.ensemble import GradientBoostingClassifier

from imblearn.pipeline import Pipeline # 파이프라인 구축 시, smote 함수를 사용할 때 불러오는 패키지
pipeline = Pipeline([('processor', column_transformer), 
                    ('over', SMOTE()), 
                    ('model', GradientBoostingClassifier())])

gb_param_grid = {'model__loss' : ["deviance"],
              'model__n_estimators' : [100,200,300],
              'model__learning_rate': [0.1, 0.05, 0.01],
              'model__max_depth': [4, 8],
              'model__min_samples_leaf': [100,150],
              'model__max_features': [0.3, 0.1] 
              }
 
gsGBC = GridSearchCV(pipeline, param_grid = gb_param_grid, cv=5, scoring="accuracy", n_jobs = -1)
 
gsGBC.fit(x_train, y_train)
 
GBC_best = gsGBC.best_estimator_
 
# Best score
gsGBC.best_score_

C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9286836879432624

In [58]:
# 그리드 서치
from imblearn.pipeline import Pipeline # 파이프라인 구축 시, smote 함수를 사용할 때 불러오는 패키지
pipeline = Pipeline([('processor', column_transformer), 
                    ('over', SMOTE()), 
                    ('model', SVC(probability=True))])

svc_param_grid = {'model__kernel': ['rbf'], 
                  'model__gamma': [ 0.001, 0.01, 0.1, 1],
                  'model__C': [1, 10, 50, 100,200,300, 1000]}
 
gsSVMC = GridSearchCV(pipeline, param_grid = svc_param_grid, cv=5, scoring="accuracy", n_jobs= -1)
 
gsSVMC.fit(x_train, y_train)
 
SVMC_best = gsSVMC.best_estimator_
 
# Best score
gsSVMC.best_score_

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9042070921985814

In [60]:
# 그리드 서치
from xgboost import XGBClassifier

from imblearn.pipeline import Pipeline # 파이프라인 구축 시, smote 함수를 사용할 때 불러오는 패키지
pipeline = Pipeline([('processor', column_transformer), 
                    ('over', SMOTE()), 
                    ('model', XGBClassifier())])

xgb_param_grid = {'model__max_depth':[3,5,7],
                  'model__min_child_weight':[3,5,6],
                  'model__gamma': [ 0, 0.001, 0.01, 0.1, 1],
                  'model__learning_rate':[0.1, 0.05, 0.01]}
 
gsXGBC = GridSearchCV(pipeline, param_grid = xgb_param_grid, cv=5, scoring="accuracy", n_jobs= -1)
 
gsXGBC.fit(x_train, y_train)
 
XGBC_best = gsXGBC.best_estimator_
 
# Best score
gsXGBC.best_score_

[14:08:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9403943262411347

In [61]:
#Voting Classifier - voting 파라미터로 hard/soft 선택가능
votingC = VotingClassifier(
    estimators=[('rfc', RFC_best), ('svc', SVMC_best),('gbc',GBC_best), ('xgb', XGBC_best)], 
    voting='soft', # 간접 투표
    n_jobs=4
)

# 모델 학습
votingC = votingC.fit(x_train, y_train)

#모델 검증
print('학습 데이터 점수 :{:.4f}'.format(votingC.score(x_train, y_train)))
print('평가 데이터 점수 :{:.4f}'.format(votingC.score(x_test, y_test)))

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


학습 데이터 점수 :0.9766
평가 데이터 점수 :0.9479


In [62]:
x_test=out.drop(['02)만성질환명(뇌졸중,중풍)'],axis=1)
x_test

,생년,가구 월평균 지출액,월 혈압약 일수,장애등급,가구 유형,가장 불편한부위,장애주된 원인,주된 진단명,혈압약 복용 여부,암 종류1,암 종류2,EQ-5D (자기관리),스마트폰 사용 여부
39,1928,100,30,6,40,36,48,6,1,0,0,3,2


In [63]:
labels = ['무', '유']

In [64]:
# x_test = transformer.transform(x_test)

y_predict = votingC.predict(x_test)
label = labels[y_predict[0]]
y_predict = votingC.predict_proba(x_test)
confidence = round(y_predict[0][y_predict[0].argmax()], 4)

print(label, confidence) 

무 0.8118
